In [24]:
import io
import os
import cv2
import random
import tqdm
import glob
import traceback
import psycopg2
import pymongo
#from gridfs import GridFS
#import bson
#from bson.objectid import ObjectId
from flask import Flask, request
from zipfile import ZipFile
from multiprocessing.pool import ThreadPool
from embeddings import get_embeddings


class MongoDB:
    def __init__(self, cred):
        self.mongo_cred = cred
        self.mongo_client = pymongo.MongoClient(self.mongo_cred)
        self.mongo_db = self.mongo_client.dummy
        self.mongo_col = self.mongo_db["data"]
        self.mongo_fs = gridfs.GridFS(self.mongo_db, collection='full_images')

    def put_to_fs(self, im_bytes, meta=False):
        # Store image to GridFS
        id_ = self.mongo_fs.put(im_bytes)
        if meta:
            # Add metadata to the file
            self.mongo_fs.put(bson.BSON.encode(meta), _id=id_, metadata=True)
        return id_


class PostGresDB:
    def __init__(self, database, user, password, host, port):
        self.conn = psycopg2.connect(
            database=database, user=user, password=password, host=host, port=port
        )
        self.conn.autocommit = True
        self.cur = self.conn.cursor()

    def insert_record(self, name, embedding, path):
        sql_query = '''insert into image_embeddings(name, embedding, path)
                       values(%s, %s, %s) RETURNING id; '''
        self.cur.execute(sql_query, (name, embedding, path))
        key_value = self.cur.fetchone()[0]
        return key_value

# def get_embeddings(im_name):
#     im = cv2.imread(im_name)
#     width = im.shape[1]
#     height = im.shape[0]
#     new_width = int(width*0.5)
#     new_height = int(height*0.5)
#     x = random.randint(0,new_width)
#     y = random.randint(0,new_height)
#     return im[y:y+new_width,x:x+new_width,:]



def handle_image(im_path, mongo_db_obj, postgres_db_obj):
    try:
        with open(im_path, 'rb') as f:
            im_bytes = f.read()
        im_name = os.path.basename(im_path).split(".")[0]
        # Get vector using get_embeddings(im_name) and store in data variable
        data = get_embeddings(im_name)
        key_value = postgres_db_obj.insert_record(im_name, data, im_path)
        data_to_store = {"id": key_value, "image_name": im_name}
        id_ = mongo_db_obj.put_to_fs(im_bytes, meta=data_to_store)
        return id_
    except:
        pass
    return False


mongo_db_obj = MongoDB('your credentials')
postgres_db_obj = PostGresDB(database='testdb', user='testdatabase', password= 'arooj242', host='localhost', port='5432')
# postgres_db_obj = PostGresDB(database='your credentials', user='your credentials', password='your credentials', host='your credentials', port='your credentials')


zips_dest_dirs = "test"

if not os.path.exists(zips_dest_dirs):
    os.mkdir(zips_dest_dirs)


app = Flask(__name__)

@app.route('/enroll', methods=['GET', 'POST'])
def enroll():
    try:
        if request.method == 'POST':
            # Extract images and person names from the zip file
            with ZipFile(request.files['archive'].stream, 'r') as zip_file:
                zip_file.extractall(z)
                
    except Exception as e:

        return {"status":str(traceback.format_exc())}


app.run(host='0.0.0.0')



NameError: name 'gridfs' is not defined